In [15]:
import os
import tarfile
import urllib
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from pandas.plotting import scatter_matrix


dr = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
housepath = os.path.join("datasets", "housing")
# print(housepath)
houseurl = dr + "datasets/housing/housing.tgz"
# print(houseurl)

def fetch_housing_data(housing_url=houseurl, housing_path=housepath):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
#     print(tgz_path)
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

def load_house_data(hp=housepath):
    csv_path = os.path.join(hp, "housing.csv")
    return pd.read_csv(csv_path)

def split_train_test(data, ratio):
    shuffled = np.random.permutation(len(data))
    testsize = int(len(data) * ratio)
    test_indices = shuffled[:testsize]
    train_indices = shuffled[testsize:]
    return data.iloc[train_indices], data.iloc[test_indices]

housing = load_house_data()
# housing.head()
# housing.info()
# housing['ocean_proximity'].value_counts()
# housing.describe()
# housing.hist(bins=50, figsize=(20,15))
# plt.show()
# traindf, testdf = split_train_test(housing, .2)
# print(len(traindf), len(testdf))
traindf, testdf = train_test_split(housing, test_size=.2, random_state=42)
# stratified sampling using pandas
# housing.info()
housing["income_cat"] = pd.cut(housing["median_income"], 
                              bins=[0.,1.5,3.0,4.5,6., np.inf],
                              labels=[1,2,3,4,5])
# rather than do it on our own, create the stratified data by calling scikit
split = StratifiedShuffleSplit(n_splits=1, test_size=.2, random_state=42)
for train_ind, test_ind in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_ind]
    strat_test_set = housing.loc[test_ind]

strat_test_set["income_cat"].value_counts() / len(strat_test_set)
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
housing = strat_train_set.copy()
# exploring data and plotting different types of scatter plots
# housing.plot(kind="scatter", x = "longitude", y = "latitude")
# alpha parameter shows the density
# housing.plot(kind="scatter", x = "longitude", y = "latitude", alpha=.1)
# housing.plot(kind = "scatter", x = "longitude", y = "latitude", alpha = .4, 
#              s = housing["population"]/100, label = "population", figsize = (10,7), 
#             c = "median_house_value", cmap = plt.get_cmap("jet"), colorbar = True,)
# plt.legend()
# create a correlation matrix and display
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending = False)
attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
# create a scatter plot matrix and display
# scatter_matrix(housing[attributes], figsize=(12,8))
# housing.plot(kind = 'scatter', x = 'median_income', y = 'median_house_value', alpha = .1)
housing['rooms_per_household'] = housing['total_rooms'] / housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms'] / housing['total_rooms']
housing['population_per_household'] = housing['population'] / housing['households']
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending = False)

housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()
imputer = SimpleImputer(strategy = 'median')
housing_num = housing.drop('ocean_proximity', axis = 1)
imputer.fit(housing_num)
# imputer.statistics_
# housing_num.median().values
x = imputer.transform(housing_num)
htr = pd.DataFrame(x, columns = housing_num.columns, index = housing_num.index)
# htr.head()
# print(type(htr))
housing_cat = housing[['ocean_proximity']]
housing_cat.head()
# scikit is always using objects, for instance
# we will use the ordinalencoder to discretize our
# categorical variables
ordinal_encoder = OrdinalEncoder()
# discretizing returns numpy array
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]
ordinal_encoder.categories_
# use one hot encoding instead because ml will determine
# that categories with shorter distance are similar
# but this is not the case with our dataset.
# one hot encoding gives a binary answer to if
# the data belongs to certain category, inland, island, etc
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
# outputs a scipy sparse matrix instead of numpy array
housing_cat_1hot
# sparse matrix only stores locations of 1 in sea of 0s.
# can convert to dense numpy array as follows
housing_cat_1hot.toarray()
cat_encoder.categories_
# if there are many categories, one hot encoding will cause
# large number of input features, can slow down performance
# could just replace categorical input with relevant feature,
# instead of this category we could put the actual distance to the ocean
# or you could replace each category iwth low dimension vector called an embedding
# each category's representation will be learned in training. It is representation learning
# chapters 13 and 17

#create new class to transform data, basically custom transformation
# attr_adder = CombinedAttributesAdder(add_bedrooms_per_room = False)
# housing_extra_attribs = attr_adder.transform(housing.values)

rooms_ix, bedrooms_ix, population_ix, households_ix = 3,4,5,6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        # no args or kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room = False)
housing_etra_attribs = attr_adder.transform(housing.values)

#instead of doing the above, we can just do pipeline of processes
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')),
                        ('attribs_adder', CombinedAttributesAdder()),
                        ('std_scaler', StandardScaler()),
                        ])
housing_num_tr = num_pipeline.fit_transform(housing_num)

# do both in one instead
num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs),
])
# this pipeline stuff is cool it reminds me of
# shell project 

housing_prepared = full_pipeline.fit_transform(housing)


# well that was easy
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

sd = housing.iloc[:5]
sl = housing_labels.iloc[:5]
sdp = full_pipeline.transform(sd)
# print('Predictions', lin_reg.predict(sdp))
# print('Labels', list(sl))
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
# the linear regression model sucks for this
# error of $68000 dollars
lin_rmse

# lets do a regression decision tree
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

def displayScores(scores):
    print('Scores: ', scores)
    print('Mean: ', scores.mean())
    print('Standard deviation: ', scores.std())

# let's use cross validation cause overfitting
# scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv = 10)
# tree_rmse_scores = np.sqrt(-scores)
# displayScores(tree_rmse_scores)
# lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv = 10)
# lin_rmse_scores = np.sqrt(-lin_scores)
# displayScores(lin_rmse_scores)
# decision tree is overfitting like crazy
# how about we use the ensemble method of random forests
forest = RandomForestRegressor()
forest.fit(housing_prepared, housing_labels)
# print('hi')
# forscore = cross_val_score(forest, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv = 10)
# forestrmse = np.sqrt(-forscore)
# displayScores(forestrmse)
# print('hello')

# use grid search to tune hyperparameters
param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2,3,4]}
]
grid_search = GridSearchCV(forest, param_grid, cv = 5, scoring = 'neg_mean_squared_error', return_train_score = True)
grid_search.fit(housing_prepared, housing_labels)
# print(grid_search.best_params_)
# print(grid_search.best_estimator_)
# cvres = grid_search.cv_results_
# for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
#     print(np.sqrt(-mean_score), params)
feature_importances = grid_search.best_estimator_.feature_importances_
extra_attribs = ['rooms_per_hhold', 'pop_per_hhold', 'bedrooms_per_room']
cat_encoder = full_pipeline.named_transformers_['cat']
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse = True)

# now evaluate on the test set
finalModel = grid_search.best_estimator_

# x is data, y are labels
XTest = strat_test_set.drop('median_house_value', axis = 1)
yTest = strat_test_set['median_house_value'].copy()

XTestPrepared = full_pipeline.transform(XTest)

finalPreds = finalModel.predict(XTestPrepared)
finalMSE = mean_squared_error(yTest, finalPreds)
finalRMSE = np.sqrt(finalMSE)

confidence = .95
squaredErrors = (finalPreds - yTest) ** 2
np.sqrt(stats.t.interval(confidence, len(squaredErrors) - 1,
                        loc = squaredErrors.mean(),
                        scale = stats.sem(squaredErrors)))

# lastly, introduced the idea of joblib to load and use the same model
# again and again. Useful if you wanted to deploy on a server or make into
# an api so you don't train a model every time

array([45890.69748766, 49731.54139789])